<a href="https://colab.research.google.com/github/Utree/deeplearning2_colab_log/blob/master/5%E7%AB%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# common/config.py
GPU = False

In [0]:
# common/np.py
if GPU:
  import cupy as np
  np.cuda.set_allocator(np.cuda.MemoryPool().malloc)
  np.add.at = np.scatter_add
  
  print('\033[92m' + '-' * 60 + '\033[0m')
  print(' ' * 23 + '\033[92mGPU Mode (cupy)\033[0m')
  print('\033[92m' + '-' * 60 + '\033[0m\n')
else:
  import numpy as np

In [0]:
# common/functions.py
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def relu(x):
  return np.maximum(0, x)

def softmax(x):
  if x.ndim == 2:
    x = x - x.max(axis=1, keepdims=True)
    x = np.exp(x)
    x /= x.sum(axis=1, keepdims=True)
  elif x.ndim == 1:
    x = x - np.max(x)
    x = np.exp(x) / np.sum(np.exp(x))
    
  return x

def cross_entropy_error(y, t):
  if y.ndim == 1:
    t = t.reshape(1, t.size)
    y = y.reshape(1, y.size)
    
  # 教師データがone-hot-vectorの場合、正解ラベルのインデックスに変換
  if t.size == y.size:
    t = t.argmax(axis=1)
    
  batch_size = y.shape[0]
  
  return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

In [0]:
# common/layers.py
class MatMul:
  def __init__(self, W):
    self.params = [W]
    self.grads = [np.zeros_like(W)]
    self.x = None
    
  def forward(self, x):
    W, = self.params
    out = np.dot(x, W)
    self.x = x
    return out
  
  def backward(self, dout):
    W, = self.params
    dx = np.dot(dout, W.T)
    dW = np.dot(self.x.T, dout)
    self.grads[0][...] = dW
    return dx
  
class Affine:
  def __init__(self, W, b):
    self.params = [W, b]
    self.grads = [np.zeros_like(W), np.zeros_like(b)]
    self.x = None
    
  def forward(self, x):
    W, b = self.params
    out = np.dot(x, W) + b
    self.x = x
    return out
  
  def backward(self, dout):
    W, b = self.params
    dx = np.dot(dout, W.T)
    dW = np.dot(self.x.T, dout)
    db = np.sum(dout, axis=0)
    
    self.grads[0][...] = dW
    self.grads[1][...] = db
    return dx
  
class Softmax:
  def __init__(self):
    self.params, self.grads = [], []
    self.out = None
    
  def forward(self, x):
    self.out = softmax(x)
    return self.out
  
  def backward(self, dout):
    dx = self.out * dout
    sumdx = np.sum(dx, axis=1, keepdims=True)
    dx -= self.out * sumdx
    return dx
  
class SoftmaxWithLoss:
  def __init__(self):
    self.params, self.grads = [], []
    self.y = None # softmaxの出力
    self.t = None # 教師ラベル
    
  def forward(self, x, t):
    self.t = t
    self.y = softmax(x)
    
    # 教師ラベルがone-hotベクトルの場合、正解のインデックスに変換
    if self.t.size == self.y.size:
      self.t = self.t.argmax(axis=1)
      
    loss = cross_entropy_error(self.y, self.t)
    return loss
  
  def backward(self, dout=1):
    batch_size = self.t.shape[0]
    
    dx = self.y.copy()
    dx[np.arange(batch_size), self.t] -= 1
    dx *= dout
    dx = dx / batch_size
    
    return dx
  
class Sigmoid:
  def __init__(self):
    self.params, self.grads = [], []
    self.out = None
    
  def forward(self, x):
    out = 1 / (1 + np.exp(-x))
    self.out = out
    return out
  
  def backward(self, dout):
    dx = dout * (1.0 - self.out) * self.out
    return dx
  
class SigmoidWithLoss:
  def __init__(self):
    self.params, self.grads = [], []
    self.loss = None
    self.y = None # sigmoidの出力
    self.t = None # 教師データ
    
  def forward(self, x, t):
    self.t = t
    self.y = 1 / (1 + np.exp(-x))
    
    self.loss = cross_entropy_error(np.c_[1 - self.y, self.y], self.t)
    
    return self.loss
  
  def backward(self, dout=1):
    batch_size = self.t.shape[0]
    
    dx = (self.y - self.t) * dout / batch_size
    return dx
  
class Dropout:
  def __init__(self, dropout_ratio=0.5):
    self.params, self.grads = [], []
    self.dropout_ratio = dropout_ratio
    self.mask = None
    
  def forward(self, x, train_flg=True):
    if train_flg:
      self.mask = np.random.rand(*x.shape) > self.dropout_ratio
      return x * self.mask
    else:
      return x * (1.0 - self.dropout_ratio)
    
  def backward(self, dout):
    return dout * self.mask
  
class Embedding:
  def __init__(self, W):
    self.params = [W]
    self.grads = [np.zeros_like(W)]
    self.idx = None
    
  def forward(self, idx):
    W, = self.params
    self.idx = idx
    out = W[idx]
    return out
  
  def backward(self, dout):
    dW, = self.grads
    dW[...] = 0
    np.add.at(dW, self.idx, dout)
    return None

In [0]:
# common/time_layers.py


- 前章までのニューラルネットワークは、フィードフォワードと呼ばれるタイプのネットワーク
- フィードフォワードとは、流れが1方向のネットワークを指す
- フィードフォワード・ネットワークでは、時系列データを上手く扱えない
- 時系列データの性質(パターン)を学習するためにRNN(リカレントニューラルネットワーク)を用いる

word2vecを確率の視点から見る

word2vecのCBOWモデルでは、コンテキストの単語からターゲットとなる単語を推測する

Wt-1 -> Wt <- Wt+1

つまり、コンテキスト: Wt-1とWt+1が与えられたとき、ターゲットがWtとなる確率を求める

これが、ウィンドウサイズ1のCBOWモデル

言語モデルは単語の並びに対して確率を与える

与えられた単語の並びがどれだけ自然な単語の並びなのかを確率で評価する

言語モデルは様々なアプリケーションで利用できる

- 人の発話からいくつかの文章を候補として生成し、文章として自然であるかをランク付けする
- 新しい文章を生成する用途にも利用でき、単語列の自然さから確率分布に従って、単語をサンプリング(紡ぎ出す)し、文章を生成する
　

数式を用いて言語モデルを記述すると、
m個の単語からなる文章は、順序を考慮した上で単語が出現する確率とみなすことができ、
同時確率 : P(W1, W2, ... Wm)と表す。

言語モデルが扱う事後確率は、t番目の単語をターゲットとして、t番目より左側の単語すべてをコンテキスト(条件)とする

では、前章で登場した、word2vecのCBOWモデルを言語モデルに適応するためには、コンテキストのサイズをある値に限定することで、近似的に表すことができる

しかし、CBOWモデルには２つの問題点がある。

- １つ目は、コンテキストの大きさを固定することで、それ以前に登場する単語を考慮することが出来ない
- 2つ目は、CBOWモデルはコンテキスト内の単語の並びが無視されてしまう

中間層の単語ベクトルを並びを考慮し、連結するアプローチもあるが、コンテキストのサイズに比例して、重みパラメータも増加してしまう

そのため、RNNを用いる

RNNとは、直訳で、循環するニューラルネットワークを意味する

RNNレイヤでは、閉じた経路、ループする経路を持つ。

ループすることで、過去のデータを記憶しながら、最新のデータへと更新することができる。

RNNレイヤでの計算は次式で表される

ht = tanh(ht-1Wh + xtWx + b)

ここのtanhは活性化関数(シグモイド関数etc..)を表しており、数学的には、双曲線接線関数という。

双曲線接線関数はシグモイド関数の線形変換(y=ax+b: 足したり、掛けたり、足したり、割ったり、幾何的には、ベクトル空間を実数倍、回転、折返ししているのと同値)

シグモイド関数では、マイナスの値を取らなかったが、双曲線接線関数では、マイナスをとる

ループを展開したあとのRNNは通常の誤差逆伝播法を使い、勾配を求める。

このとき、時間方向に展開したニューラルネットワークの誤差逆伝播法という意味で、BPTT(Backpropagation Through Time)と呼ぶ。

長い時系列データを学習するとき、問題が起こる。

長い時系列データでは、BPTTの計算リソースも大きくなるため、勾配が不安定(レイヤが長くなると、勾配が小さくなることがあり、前時刻へと届かなくなる)になり、メモリ使用量も増加する。

この解決策として、逆伝播のネットワークのつながりを断ち切る Truncated BPTTという手法が用いられる。
(順伝播では、途切れることはない)

RNNでは、データをシーケンシャル(順番)に与える必要がある

シーケンシャルにデータを与えるとき、バッチ処理では、開始位置を各バッチでずらす必要がある。

また、データが途中で終端に達した場合は先頭に戻す対応が必要

これまで循環するニューラルネットワークであるRNNレイヤ、時系列データをまとめて処理するTime RNNレイヤを実装してきました。

次は、時系列データを扱い、最後に損失を評価するRNNLMを実装する

RNNLMで最終的な損失を求めるとき、平均値を出す。

RNNLMの重みとバイアスを初期化する際、前奏からのノード数を用いて、1/ √nの標準偏差を持つ文武で初期化する

## 言語モデルの評価

言語モデルの予測性能の良さを評価する指標として、**パープレキシティ** がよく用いられる

## パープレキシティとは
確率の逆数

直感的には、分岐数と解釈でき、次に取りうる選択肢の数とみなせる